In [1]:
#first it decodes the message and writes it as a text file:
import whisper

model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large

result = model.transcribe("command_test.mp3")
print(result["text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:04<00:00, 16.2MiB/s]
C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 test test test I'm gonna be talking like this let's see how it handles when it's super loud and then also when I'm really close and it's super loud or when I'm pretty far away and it's kind of quiet let's see if it can handle a little bit of this a little bit of that some flaps and I think birthday party power off run function meeting at 5 p.m. in new Plaza Square you need a doctor's appointment pretty soon make sure you get that going don't forget to visit the birthday party at 7 p.m.


first lets see if it can do this in real time. 
to do this, since the model is trained on 30 second intervals we will need to break the audio up into 30 second chunks.
then while its transcribing one section, it will be recording the next section, that way its going to be at most 30 seconds behind.
it will save these 30 second chunks to a file, labeled with date and time and then it will search through the file for any new information


In [ ]:
import threading
import os
import numpy as np
import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import time
from datetime import datetime

model = whisper.load_model("medium")
file_lock = threading.Lock()

duration = 30  # seconds
samplerate = 16000
stop_recording = False

recording_buffer = {
    "Thread-1": None,
    "Thread-2": None
}

def record_audio(thread_name):
    print(f"{thread_name}: Recording for {duration}s...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    return recording

def save_and_transcribe(thread_name, audio_data):
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    wav_filename = f"{thread_name}_{current_datetime}.wav"
    mp3_filename = f"{thread_name}_{current_datetime}.mp3"

    write(wav_filename, samplerate, audio_data)
    audio = AudioSegment.from_wav(wav_filename)
    audio.export(mp3_filename, format="mp3")
    os.remove(wav_filename)

    print(f"{thread_name}: Transcribing...")
    result = model.transcribe(mp3_filename)
    clean_text = result["text"].replace('\n', ' ').replace('\r', ' ').strip()

    with file_lock:
        with open("transcription.txt", "a", encoding="utf-8") as f:
            f.write(f"[{datetime.now()}][{thread_name}] {clean_text}\n")

    os.remove(mp3_filename)
    print(f"{thread_name}: Transcription complete and file cleaned up.")

def worker(my_name, partner_name):
    global stop_recording
    while not stop_recording:
        # Record audio
        audio_data = record_audio(my_name)
        # Store for the partner to process later (if needed)
        recording_buffer[my_name] = audio_data

        # Allow partner to record while this thread processes previous data
        if recording_buffer[partner_name] is not None:
            save_and_transcribe(partner_name, recording_buffer[partner_name])
            recording_buffer[partner_name] = None
        else:
            # If partner hasn't recorded yet, process own data
            save_and_transcribe(my_name, audio_data)
            recording_buffer[my_name] = None

def start_alternating_threads():
    thread1 = threading.Thread(target=worker, args=("Thread-1", "Thread-2"))
    thread2 = threading.Thread(target=worker, args=("Thread-2", "Thread-1"))

    thread1.start()
    # Stagger the second thread to alternate
    time.sleep(duration)
    thread2.start()

    return thread1, thread2

try:
    thread1, thread2 = start_alternating_threads()
    runtime = 5 * 60  # Run for 5 minutes, adjust as needed
    time.sleep(runtime)
    stop_recording = True
    thread1.join()
    thread2.join()
    print("Recording and transcription stopped.")
except KeyboardInterrupt:
    stop_recording = True
    print("Stopped by user.")


Recorder: Recording audio...
Main: Running...


C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Exception in thread Thread-4 (processing_thread):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 994, in run
    self._target(*self._args, **s

Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Transcriber: Transcribing audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Audio added to queue.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Recorder: Finished recording.
Recorder: Queue is full, skipping this recording.
Recorder: Recording audio...
Main: Interrupted by user.


In [ ]:
#code words and new information
# first it will look throught the transcript for any new information.
# it will do this any time that there is a break in the transcription.
# every 5 minutes it will check through the transcript to see if there is a break with no new text
# if there isnt, then it will look through from the last time it checked, up to the current time.



In [1]:
import threading
import os
import numpy as np
import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import time
from datetime import datetime

model = whisper.load_model("medium")
file_lock = threading.Lock()

duration = 30  # seconds
samplerate = 16000
stop_recording = False

recording_buffer = {
    "Thread-1": None,
    "Thread-2": None
}

def record_audio(thread_name):
    print(f"{thread_name}: Recording for {duration}s...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    return recording

def save_and_transcribe(thread_name, audio_data):
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    wav_filename = f"{thread_name}_{current_datetime}.wav"
    mp3_filename = f"{thread_name}_{current_datetime}.mp3"

    write(wav_filename, samplerate, audio_data)
    audio = AudioSegment.from_wav(wav_filename)
    audio.export(mp3_filename, format="mp3")
    os.remove(wav_filename)

    print(f"{thread_name}: Transcribing...")
    result = model.transcribe(mp3_filename)
    clean_text = result["text"].replace('\n', ' ').replace('\r', ' ').strip()

    with file_lock:
        with open("transcription.txt", "a") as f:
            f.write(f"[{datetime.now()}][{thread_name}] {clean_text}\n")

    os.remove(mp3_filename)
    print(f"{thread_name}: Transcription complete and file cleaned up.")

def worker(my_name, partner_name):
    global stop_recording
    while not stop_recording:
        # Record audio
        audio_data = record_audio(my_name)
        # Store for the partner to process later (if needed)
        recording_buffer[my_name] = audio_data

        # Allow partner to record while this thread processes previous data
        if recording_buffer[partner_name] is not None:
            save_and_transcribe(partner_name, recording_buffer[partner_name])
            recording_buffer[partner_name] = None
        else:
            # If partner hasn't recorded yet, process own data
            save_and_transcribe(my_name, audio_data)
            recording_buffer[my_name] = None

def start_alternating_threads():
    thread1 = threading.Thread(target=worker, args=("Thread-1", "Thread-2"))
    thread2 = threading.Thread(target=worker, args=("Thread-2", "Thread-1"))

    thread1.start()
    # Stagger the second thread to alternate
    time.sleep(duration)
    thread2.start()

    return thread1, thread2

try:
    thread1, thread2 = start_alternating_threads()
    runtime = 5 * 60  # Run for 5 minutes, adjust as needed
    time.sleep(runtime)
    stop_recording = True
    thread1.join()
    thread2.join()
    print("Recording and transcription stopped.")
except KeyboardInterrupt:
    stop_recording = True
    print("Stopped by user.")


Thread-1: Recording for 30s...
Thread-2: Recording for 30s...
Thread-1: Transcribing...


C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
Thread-2: Transcription complete and file cleaned up.
Thread-2: Recording for 30s...
Thread-1: Transcribing...
Thread-1: Transcription complete and file cleaned up.
Thread-1: Recording for 30s...
Thread-2: Transcribing...
T

: 